In [2]:
import re
import torch
import json

from seqeval.metrics import accuracy_score, f1_score, classification_report, precision_score, recall_score
from seqeval.scheme import BILOU
from tokenizers import decoders, Encoding
from transformers import BertForTokenClassification, BertTokenizer, BertTokenizerFast, BatchEncoding

from ner_pipeline.alignment import align_tokens_and_annotations_bilou
from ner_pipeline.prepare_data import prepare_data
from ner_pipeline.labelset import LabelSet

In [3]:
model = BertForTokenClassification.from_pretrained("bert_ner_finetuned_iliad-with-gpu-pattern2.model")

404 Client Error: Not Found for url: https://huggingface.co/bert_ner_finetuned_iliad-with-gpu-pattern2.model/resolve/main/config.json


OSError: Can't load config for 'bert_ner_finetuned_iliad-with-gpu-pattern2.model'. Make sure that:

- 'bert_ner_finetuned_iliad-with-gpu-pattern2.model' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'bert_ner_finetuned_iliad-with-gpu-pattern2.model' is the correct path to a directory containing a config.json file



In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

In [ ]:
label_list = ['O', 'B-CLEntity', 'I-CLEntity', 'L-CLEntity', 'U-CLEntity']

In [ ]:
odyssey_lines = []
odyssey = open("example-texts/odyssey.txt")
for line in odyssey:
    line = line.strip()
    odyssey_lines.append(line)

In [ ]:
odyssey_lines = [line for line in odyssey_lines if line]

In [ ]:
odyssey_lines[:5]

In [ ]:
# pred = []
# for line in odyssey_lines:
#     tokens_line = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(line)))
#     inputs_line = tokenizer.encode(line, return_tensors="pt")
#     outputs_line = model(inputs_line).logits
#     predictions_line = torch.argmax(outputs_line, dim=2)
#     pred_line_label = []
#     for prediction in predictions_line[0].numpy():
#         pred_line_label.append(label_list[prediction])
#     pred.append(pred_line_label)
    
# with open('odyssey_ner_predictions.txt', 'w') as f:
#     f.write(json.dumps(pred))

In [ ]:
with open('odyssey_ner_predictions.txt', 'r') as f:
    pred = json.loads(f.read())

In [ ]:
# Evaluation 
pattern = r'(\b[A-Z][a-z]+\b)(\s\b[A-Z][a-z]+\b)*'
odyssey_regex_matches = prepare_data("../example-texts/odyssey.txt", pattern)

In [ ]:
odyssey_regex_matches[:5] 

In [ ]:
clentity_label_set = LabelSet(labels=["CLEntity"])

In [ ]:
true = []
for match in odyssey_regex_matches:
    
    match_tokenized_batch : BatchEncoding = tokenizer(match["content"])
    match_tokenized_text : Encoding = match_tokenized_batch[0]
    aligned_label_ids = clentity_label_set.get_aligned_label_ids_from_annotations(
        match_tokenized_text, match["annotations"]
    )
    match_tokens = match_tokenized_text.tokens
    
    true_line_label = []
    for match_id in aligned_label_ids:
        true_line_label.append(label_list[match_id])
    true.append(true_line_label)

In [ ]:
print(len(true))
print(len(pred))

In [ ]:
print(classification_report(true, pred, mode='strict', scheme=BILOU))